# SeenItemsCount
* returns the number of items a given user has seen

In [ ]:
medium = ""

In [ ]:
import NBInclude: @nbinclude
@nbinclude("../Alpha.ipynb");

In [ ]:
using DataFrames

In [ ]:
function get_dataframe(split, content)
    df = get_split(split, "all", content, medium)
    DataFrame(user = df.user, item = df.item, rating = df.rating)
end;

In [ ]:
function make_prediction(users, items, u)
    r = zeros(Float32, length(users))
    Threads.@threads for i = 1:length(r)
        if users[i] in keys(u)
            r[i] = u[users[i]]
        end
    end
    r
end;

In [ ]:
for content in ["explicit", "implicit", "ptw"]
    @info "processing $content watches"
    training = get_dataframe("training", content)
    u = combine(groupby(training, :user), :rating => length => :rating)
    u = Dict(Pair.(u.user, u.rating))
    model(users, items) = make_prediction(users, items, u)
    name = "$medium/all/$(uppercasefirst(content))ItemCount"
    write_params(Dict("u" => u), name)
    write_alpha(model, medium, name; log = false)
end